In [30]:
!pip install openai
!pip install pinecone-client
!pip install langchain
!pip install unstructured

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
!git clone https://github.com/wangcheng135/bencao

fatal: destination path 'bencao' already exists and is not an empty directory.


In [74]:
!ls -alt bencao

total 32
drwxr-xr-x 6 root root 4096 Apr  9 18:00 .
drwxr-xr-x 8 root root 4096 Apr  9 18:00 .git
-rw-r--r-- 1 root root   66 Apr  9 18:00 .gitattributes
-rw-r--r-- 1 root root   18 Apr  9 18:00 README.md
drwxr-xr-x 2 root root 4096 Apr  9 18:00 renshoutangyaojing
drwxr-xr-x 2 root root 4096 Apr  9 18:00 riyongbencao
drwxr-xr-x 2 root root 4096 Apr  9 18:00 yaoxinghuiyuan
drwxr-xr-x 1 root root 4096 Apr  9 18:00 ..


In [34]:
from langchain.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader

In [75]:
def load_all_courses(solidity_root):
  loader = DirectoryLoader(solidity_root, glob = "**/readme.md")
  docs = loader.load()

  return docs

In [76]:
docs = load_all_courses("./bencao/")

In [77]:
print (f'You have {len(docs)} document(s) in your data')
print (f'There are {len(docs[0].page_content)} characters in your document')

You have 3 document(s) in your data
There are 7191 characters in your document


In [79]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [80]:
print (f'Now you have {len(split_docs)} documents')

Now you have 187 documents


In [81]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [82]:
OPENAI_API_KEY = ''
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

In [83]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name = "quickstart"

In [14]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.2 MB/s eta 0:00:00


In [84]:
docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name=index_name)

In [85]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [129]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")
query = "梅得春是哪里人？"
docs = docsearch.similarity_search(query, 3, include_metadata=True)

In [130]:
print(docs[0])

page_content='作者梅得春，字元实，钱塘（今浙江杭州）人。从其书序言可以推知，梅氏的主要医学活动在明万历年问。据记载梅氏“才如操割，谭若悬河”，具有丰富的学识和口才。和当时许多知识分子一样，梅氏也曾有过仕途之梦，但“' metadata={}


In [125]:
import openai

In [126]:
answer1 = docs[0]

In [127]:
start_sequence = "\nA:"
restart_sequence = "\n\nQ: "
openai.api_key = OPENAI_API_KEY
response1 = openai.Completion.create(
  model="text-davinci-003",
  prompt=str(answer1)+str(query),
  temperature=0,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

answer2 = response1.choices[0].text.strip()

In [131]:
print("问题:", query)
print("答案:", answer2)

问题: 梅得春是哪里人？
答案: 梅得春是钱塘（今浙江杭州）人。
